# <랭체인LangChain 노트> RAG

In [1]:
import pandas as pd

# CSV 파일을 pandas DataFrame으로 로드합니다.
df = pd.read_csv('data_list_with_content.csv')

# 'content' 열의 NaN 값을 빈 문자열로 대체합니다.
df['content'] = df['content'].fillna('')
df

,title,source,content
0,<랭체인LangChain 노트> - LangChain 한국어 튜토리얼🇰🇷,https://wikidocs.net//book/14314,
1,CH01 LangChain 시작하기,https://wikidocs.net/233341,# CH01 LangChain 시작하기\n\nLangChain은 언어 모델을 활용해...
2,01. OpenAI API 키 발급 및 테스트,https://wikidocs.net/233342,# 01. OpenAI API 키 발급 및 테스트\n\n## OpenAI API 키...
3,02. OpenAI API 사용(GPT-4o 멀티모달),https://wikidocs.net/233343,# 02. OpenAI API 사용(GPT-4o 멀티모달)\n\n```\nCopy#...
4,03. LangChain Expression Language(LCEL),https://wikidocs.net/233344,# 03. LangChain Expression Language(LCEL)\n\n#...
...,...,...,...
102,CH15 파인튜닝(Fine Tuning),https://wikidocs.net/233783,# CH15 파인튜닝(Fine Tuning)\n\n파인튜닝
103,CH16 사례(Use Cases),https://wikidocs.net/233784,# CH16 사례(Use Cases)\n\nuse cases
104,01. 도구를 활용한 토론 에이전트(Two Agent Debates with Tools),https://wikidocs.net/234162,# 01. 도구를 활용한 토론 에이전트(Two Agent Debates with T...
105,CH17 LangGraph,https://wikidocs.net/233785,# CH17 LangGraph\n\nLangGraph


In [2]:

from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

# 문서 리스트를 생성합니다.
documents = []
for index, row in df.iterrows():
    if pd.isna(row['content']):
        continue
    chunks = text_splitter.split_text(row['content'])
    for chunk in chunks:
        documents.append(Document(page_content=chunk, metadata={"title": row['title'], "source": row['source']}))

print(f"분할된 테디노트 파일의 개수: {len(documents)}")

분할된 테디노트 파일의 개수: 825


In [3]:
# langchain_openai와 langchain의 필요한 모듈들을 가져옵니다.
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

# 로컬 파일 저장소를 사용하기 위해 LocalFileStore 인스턴스를 생성합니다.
# './cache/' 디렉토리에 데이터를 저장합니다.
store = LocalFileStore("./cache/")

# OpenAI 임베딩 모델 인스턴스를 생성합니다. 모델명으로 "text-embedding-3-small"을 사용합니다.
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small", disallowed_special=())

# CacheBackedEmbeddings를 사용하여 임베딩 계산 결과를 캐시합니다.
# 이렇게 하면 임베딩을 여러 번 계산할 필요 없이 한 번 계산된 값을 재사용할 수 있습니다.
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, store, namespace=embeddings.model
)

In [4]:
# langchain_community 모듈에서 FAISS 클래스를 가져옵니다.
from langchain_community.vectorstores import FAISS

# 로컬에 저장할 FAISS 인덱스의 폴더 이름을 지정합니다.
FAISS_DB_INDEX = "teddynote_faiss"

# combined_documents 문서들과 cached_embeddings 임베딩을 사용하여 FAISS 데이터베이스 인스턴스를 생성합니다.
db = FAISS.from_documents(documents, cached_embeddings)

# 생성된 데이터베이스 인스턴스를 지정한 폴더에 로컬로 저장합니다.
db.save_local(folder_path=FAISS_DB_INDEX)

In [5]:
# FAISS 클래스의 load_local 메서드를 사용하여 저장된 벡터 인덱스를 로드합니다.
db = FAISS.load_local(
    FAISS_DB_INDEX,  # 로드할 FAISS 인덱스의 디렉토리 이름
    cached_embeddings,  # 임베딩 정보를 제공
    allow_dangerous_deserialization=True,  # 역직렬화를 허용하는 옵션
)

In [6]:
# MMR을 사용하여 검색을 수행하는 retriever를 생성합니다.
faiss_retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 10})

In [7]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 20년차 AI 개발자입니다. 당신의 임무는 주어진 질문에 대하여 최대한 문서의 정보를 활용하여 답변하는 것입니다.
문서는 Python 코드에 대한 정보를 담고 있습니다. 따라서, 답변을 작성할 때에는 Python 코드에 대한 상세한 code snippet을 포함하여 작성해주세요.
최대한 자세하게 답변하고, 한글로 답변해 주세요. 주어진 문서에서 답변을 찾을 수 없는 경우, "문서에 답변이 없습니다."라고 답변해 주세요.
답변은 출처(source)를 반드시 표기해 주세요.

#참고문서:
{context}

#질문:
{question}

#답변: 

출처:
- source1
- source2
- ...                             
"""
)

In [8]:
from langchain.callbacks.base import BaseCallbackHandler
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.callbacks.manager import CallbackManager
from langchain_core.runnables import ConfigurableField
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain_community.chat_models import ChatOllama
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic


class StreamCallback(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs):
        print(token, end="", flush=True)


llm = ChatOpenAI(
    model="gpt-4-turbo-preview",
    temperature=0,
    streaming=True,
    callbacks=[StreamCallback()],
).configurable_alternatives(
    # 이 필드에 id를 부여합니다.
    # 최종 실행 가능한 객체를 구성할 때, 이 id를 사용하여 이 필드를 구성할 수 있습니다.
    ConfigurableField(id="llm"),
    # 기본 키를 설정합니다.
    default_key="gpt4",
    claude=ChatAnthropic(
        model="claude-3-opus-20240229",
        temperature=0,
        streaming=True,
        callbacks=[StreamCallback()],
    ),
    gpt3=ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0,
        streaming=True,
        callbacks=[StreamCallback()],
    ),
    ollama=ChatOllama(
        model="EEVE-Korean-10.8B:long",
        callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    ),
)

In [9]:
# 체인을 생성합니다.
rag_chain = (
    {"context": faiss_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [10]:
answer = rag_chain.with_config(configurable={"llm": "gpt4"}).invoke(
    "PromptTemplate 사용방법을 알려주세요"
)

PromptTemplate을 사용하는 방법에 대해 설명드리겠습니다. PromptTemplate은 특정 형식의 문자열 템플릿을 정의하고, 이를 기반으로 동적으로 문자열을 생성할 수 있게 해주는 클래스입니다. 주로 변수를 포함한 문자열 템플릿을 정의하고, 이 변수들에 값을 할당하여 최종 문자열을 생성하는 데 사용됩니다.

### 기본 사용법

1. **PromptTemplate 정의하기**

   먼저, 변수를 포함한 문자열 템플릿을 정의합니다. 변수는 중괄호 `{}`로 묶어서 표시합니다.

   ```python
   from langchain_core.prompts import PromptTemplate
   
   template = "{country}의 수도는 어디인가요?"
   ```

2. **PromptTemplate 객체 생성하기**

   정의한 템플릿 문자열을 사용하여 PromptTemplate 객체를 생성합니다. 이 때, `from_template` 메소드를 사용할 수 있습니다.

   ```python
   prompt = PromptTemplate.from_template(template)
   ```

3. **변수에 값 할당하기**

   생성된 PromptTemplate 객체에 변수에 해당하는 값을 할당합니다. 이를 위해 `format` 메소드를 사용하고, 변수명을 키워드 인자로 전달합니다.

   ```python
   prompt = prompt.format(country="대한민국")
   ```

4. **최종 문자열 생성하기**

   변수에 값이 할당된 후, PromptTemplate 객체는 최종 문자열을 포함하게 됩니다. 이 문자열은 PromptTemplate 객체를 문자열로 변환하거나 출력함으로써 확인할 수 있습니다.

   ```python
   print(prompt)  # '대한민국의 수도는 어디인가요?'
   ```

### 추가 기능

- **변수 유효성 검사**

  PromptTemplate을 생성할 때 `i